# Dataset Cleaning
To compare two pedestrian estimations with different methods - field count vs. cnn estimations, clean up and prepare a dataset from two sources.

In [1]:
import pandas as pd
import altair as alt
import numpy as np

alt.data_transformers.disable_max_rows()
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)

# Get NYC field count 

metadata - http://www.nyc.gov/html/dot/downloads/pdf/bi-annual-ped-count-readme.pdf

It has three different sorts of count
* \_am: weekday morning (7-9am)
* \_pm: weekday afternoon (4-7pm)
* \_md: weekend midday (12-2pm)

In [2]:
query = ("https://data.cityofnewyork.us/resource/cqsj-cfgu.json")
df_wide = pd.read_json(query)
rename = dict()
for cname in df_wide.filter(regex='_am|_md|_pm').columns:
    rename[cname] = 'count-' + cname
df_wide = df_wide.rename(columns=rename)
df_wide.head()

,borough,the_geom,objectid,loc,street_nam,from_stree,to_street,index,count-may_07_am,count-may_07_pm,count-may_07_md,count-sept_07_am,count-sept_07_pm,count-sept_07_md,count-may_08_am,count-may_08_pm,count-may_08_md,count-sept_08_am,count-sept_08_pm,count-sept_08_md,count-may_09_am,count-may_09_pm,count-may_09_md,count-sept_09_am,count-sept_09_pm,count-sept_09_md,count-may_10_am,count-may_10_pm,count-may_10_md,count-sept_10_am,count-sept_10_pm,count-sept_10_md,count-may_11_am,count-may_11_pm,count-may_11_md,count-sept_11_am,count-sept_11_pm,count-sept_11_md,count-may_12_am,count-may_12_pm,count-may_12_md,count-sept_12_am,count-sept_12_pm,count-sept_12_md,count-may_13_am,count-may_13_pm,count-may_13_md,count-sept_13_am,count-sept_13_pm,count-sept_13_md,count-may_14_am,count-may_14_pm,count-may_14_md,count-sept_14_am,count-sept_14_pm,count-sept_14_md,count-may_15_am,count-may_15_pm,count-may_15_md,count-sept_15_am,count-sept_15_pm,count-sept_15_md,count-may_16_am,count-may_16_pm,count-may_16_md,count-sept_16_am,count-sept_16_pm,count-sept_16_md,count-may_17_am,count-may_17_pm,count-may_17_md,count-sept_17_am,count-sept_17_pm,count-sept_17_md,count-may_18_am,count-may_18_pm,count-may_18_md,count-sept_18_pm,count-sept_18_md,count-may_19_am,count-may_19_pm,count-may_19_md
0,Bronx,"{'type': 'Point', 'coordinates': [-73.90459140...",1,1,Broadway,West 231st Street,Naples Terrace,N,1189,4094,2508,734,2646,2939,802,4015,2631,1125,4310,3420,1001,3475,2832,991,4262,2469,1010,3609,3128,863,4119,2217,997,4440,2687,1328,3820,2428,1288,3328,3365,"1,268","4,315","2,276","1,210","4,710","3,825","1,206","4,590","3,008","1,220","4,384","2,641","1,450","4,646","2,996","1,788","4,980","3,033",1204,4520,2999,"1,246","4,531","2,686","1,309","3,642","2,830","1,916","5,893","2,776","1,111","4,044","2,731","1,271","4,502","2,899","4,464","2,967",-,-,-
1,Bronx,"{'type': 'Point', 'coordinates': [-73.92188432...",2,2,East 161st Street,Grand Concourse,Sheridan Avenue,Y,1511,3184,1971,1855,3754,1183,1136,2638,1522,1939,3283,1383,1351,3111,1304,1227,3137,2762,2077,3283,1409,1007,3069,1477,1734,3333,1772,2051,3525,1752,1233,1875,1912,"2,113","4,099","1,970","2,278","4,215","2,288","2,071","3,890","1,832","2,206","4,363","2,315","1,949","4,435","2,388","2,318","4,589","2,483",2005,4790,2512,"2,053","4,721","2,311","2,109","5,485","2,548","1,848","4,920","2,143","2,389","5,952","2,832","1,749","5,148","2,156","4,723","1,604","1,702","4,347","1,576"
2,Bronx,"{'type': 'Point', 'coordinates': [-73.89535781...",3,3,East Fordham Road,Valentine Avenue,Tiebout Avenue,Y,1832,12311,14391,1829,9215,15065,1061,8013,7927,2497,10010,8013,1689,8374,7472,2387,10655,6614,2285,10943,10424,2807,12658,7714,2289,15890,9167,2679,13548,8140,2125,7635,7022,"2,704","10,486","7,421","2,710","12,723","8,872","2,651","10,613","8,241","2,699","12,490","6,339","2,752","10,408","8,266","2,382","11,216","7,574",2733,10586,9072,"2,540","11,272","8,422","2,422","11,145","9,775","2,557","12,125","6,499","2,783","12,388","7,076","2,209","9,634","7,066","8,931","6,212","1,625","11,739","7,468"
3,Bronx,"{'type': 'Point', 'coordinates': [-73.87892467...",4,4,East Gun Hill Road,Bainbridge Avenue,Rochambeau Avenue,N,764,2673,1798,1360,1975,872,2102,3206,1989,3020,3915,1662,1536,2997,1672,2097,3135,1591,2954,3962,1717,1622,3006,1344,1661,2844,1510,1996,2836,1587,1906,2066,1261,"2,371","3,500","1,620","2,052","3,981","1,979","2,373","3,660","1,914","2,096","3,378","1,333","2,382","3,425","1,339","1,757","3,561","1,714",2256,2923,1574,"2,066","2,944","1,752","2,017","2,771","1,459","2,125","3,130","1,123","2,453","3,429","1,551","1,648","2,892","1,323","2,682","1,693",-,-,-
4,Bronx,"{'type': 'Point', 'coordinates': [-73.88956389...",5,5,East Tremont Avenue,Prospect Avenue,Clinton Avenue,N,650,2538,2334,1078,2739,2343,553,2438,1652,756,3095,1374,684,2660,2604,912,3637,2339,1075,5823,4085,1012,5785,3758,1093,6119,4274,1049,5859,3934,627,1503,2371,959,"3,140","2,273","1,007","4,175","2,854",760,"3,488","2,8

## Missing values

In [3]:
# Change missing value mark from '-' to NaN for easier handling
df_wide = df_wide.replace(to_replace = "-", value = np.nan)  

### missing value count - row-wise

In [4]:
# show me top 10
df_wide.isnull().sum(axis=1).sort_values(ascending=False).head(10)

101    19
28      9
113     4
103     4
86      4
88      4
112     4
100     4
102     4
98      4
dtype: int64

In [5]:
# 101 has 19 missing values. what is it?
df_wide.iloc[101]

borough                                            East River Bridges
the_geom            {'type': 'Point', 'coordinates': [-73.99094199...
objectid                                                          102
loc                                                               102
street_nam                              Manhattan Bridge - North Path
from_stree                                                   midpoint
to_street                                                         NaN
index                                                               N
count-may_07_am                                                   NaN
count-may_07_pm                                                   NaN
count-may_07_md                                                   NaN
count-sept_07_am                                                  NaN
count-sept_07_pm                                                  NaN
count-sept_07_md                                                  NaN
count-may_08_am     

In [6]:
# it is a bridge location. let's remove it.
df_wide = df_wide.drop(df_wide.index[101])

### missing value count - column-wise

In [7]:
df_wide.isnull().sum(axis=0).sort_values(ascending=False).head(15)

count-may_19_md     63
count-may_19_am     63
count-may_19_pm     63
to_street           17
count-sept_08_md     1
count-sept_08_pm     1
count-may_07_am      1
count-may_07_pm      1
count-may_07_md      1
count-sept_08_am     1
count-sept_10_md     0
count-may_10_am      0
count-may_10_pm      0
count-may_10_md      0
count-sept_10_am     0
dtype: int64

In [8]:
# 2019 data (may_19_*) has too many missing values. Let's remove them.
# also remove 08 and 07 since they have missing values and too old.
df_wide = df_wide.drop(columns=[
    'count-may_19_am',  'count-may_19_pm',  'count-may_19_md',
    'count-may_07_am',  'count-may_07_pm',  'count-may_07_md',
    'count-sept_07_am', 'count-sept_07_pm', 'count-sept_07_md',
    'count-may_08_am',  'count-may_08_pm',  'count-may_08_md',
    'count-sept_08_am', 'count-sept_08_pm', 'count-sept_08_md'
])

## Convert to long format

In [9]:
df_long = pd.wide_to_long(df_wide, 
                          stubnames='count',
                          i='loc', 
                          j='time_measured', 
                          suffix='\w+', 
                          sep='-').reset_index()
df_long.head()

,loc,time_measured,the_geom,borough,objectid,from_stree,index,street_nam,to_street,count
0,1,may_09_am,"{'type': 'Point', 'coordinates': [-73.90459140...",Bronx,1,West 231st Street,N,Broadway,Naples Terrace,1001
1,2,may_09_am,"{'type': 'Point', 'coordinates': [-73.92188432...",Bronx,2,Grand Concourse,Y,East 161st Street,Sheridan Avenue,1351
2,3,may_09_am,"{'type': 'Point', 'coordinates': [-73.89535781...",Bronx,3,Valentine Avenue,Y,East Fordham Road,Tiebout Avenue,1689
3,4,may_09_am,"{'type': 'Point', 'coordinates': [-73.87892467...",Bronx,4,Bainbridge Avenue,N,East Gun Hill Road,Rochambeau Avenue,1536
4,5,may_09_am,"{'type': 'Point', 'coordinates': [-73.88956389...",Bronx,5,Prospect Avenue,N,East Tremont Avenue,Clinton Avenue,684


## Split time_measured 
to year-month (datetime), and time (category: md/am/pm)

In [10]:
df_long['time_measured'] = df_long['time_measured'].replace(to_replace=r'^may_', value='05_', regex=True)
df_long['time_measured'] = df_long['time_measured'].replace(to_replace=r'^sept_', value='09_', regex=True)

split = df_long['time_measured'].str.rsplit(pat='_', n=1, expand = True)

df_long['measure_month'] = split[0]
df_long['measure_hour'] = split[1]
df_long['measure_month'] = pd.to_datetime(df_long['measure_month'], format="%m_%y")
df_long = df_long[['loc', 'measure_month', 'measure_hour', 'count']]
df_long['count'] = pd.to_numeric(df_long['count'].astype(str).str.replace(',',''), errors="coerce")
df_long.head()

,loc,measure_month,measure_hour,count
0,1,2009-05-01,am,1001
1,2,2009-05-01,am,1351
2,3,2009-05-01,am,1689
3,4,2009-05-01,am,1536
4,5,2009-05-01,am,684


In [11]:
df_long.to_csv("../data/dataframe/nyc-field-count.csv", index=False)

In [12]:
len(df_long['loc'].unique())

113

## describing field count data

In [13]:
alt.Chart(df_long).mark_bar().encode(
    alt.X("count", 
          bin=alt.Bin(maxbins=100), 
          title='Field Count', 
          axis=alt.Axis(labelAngle=-45),
          scale=alt.Scale(domain=(0, 40000))),
    alt.Y('count()', title='Observations')
).properties(
    width=600, 
    height=150
).facet(
    row='measure_hour:N'
)

alt.FacetChart(...)

In [14]:
alt.Chart(df_long).mark_line().encode(
    alt.X("measure_month"),
    alt.Y('mean(count)'),
    color='measure_hour'
).properties(
    width=600, 
    height=150
)

alt.Chart(...)

## Note.
- Note that the NYC datatset has no entry for Sep. 2018 AM counts.
- What happend in May 2012 PM?

## Aggregation
We have multiple data points for 113 locations (remember that we dropeed one location - loc 102) measured from 2010 to 2018, May and September, and AM/MD/PM. We need to reduced muiltiple data points into one value  per each location so that we can compare the value to CNN based estimation. 

We are going to use 1) 2018 May and September average and 2) 3-year average (May/September 2016, May/September 2017, May/September 2018) since the GSVs are mostly taken between 2018 and 2019. 

Also, we don't know which hour is the best fit among AM/MD/PM. We are going to use the three AM/MD/PM and average of the three as well. 

To sum up,

(2018, 3-year) * (AM, MD, PM, ALL) gives us eight aggregated values for each location.


### 2018 one year

In [15]:
# average for am/md/pm
df_1year = df_long[df_long['measure_month'].between('2018-01-01', '2018-12-31')]\
    .groupby(["loc", "measure_hour"]).mean().unstack(level=1)
df_1year.columns = ['_'.join(col) +'_2018' for col in df_1year.columns]
# average across am/md/pm
df_1year['count_all_2018'] = df_long[df_long['measure_month'].between('2018-01-01', '2018-12-31')]\
    .groupby(["loc"]).mean()
df_1year.head()

,count_am_2018,count_md_2018,count_pm_2018,count_all_2018
loc,,,,
1,1271.0,2933.0,4483.0,3220.6
2,1749.0,1880.0,4935.5,3076.0
3,2209.0,6639.0,9282.5,6810.4
4,1648.0,1508.0,2787.0,2047.6
5,1016.0,2513.0,3771.0,2716.8


### 2016-2018 3-year

In [16]:
# average for am/md/pm
df_3year = df_long[df_long['measure_month'].between('2016-01-01', '2018-12-31')]\
    .groupby(["loc", "measure_hour"]).mean().unstack(level=1)
df_3year.columns = ['_'.join(col) +'_1618' for col in df_3year.columns]
# average across am/md/pm
df_3year['count_all_1618'] = df_long[df_long['measure_month'].between('2016-01-01', '2018-12-31')]\
    .groupby(["loc"]).mean()
df_3year.head()

,count_am_1618,count_md_1618,count_pm_1618,count_all_1618
loc,,,,
1,1370.6,2814.833333,4512.666667,2989.294118
2,2029.6,2265.666667,5158.166667,3217.117647
3,2502.2,7508.333333,10915.833333,7238.588235
4,2061.8,1483.500000,2974.666667,2179.882353
5,1034.4,2504.000000,3588.833333,2454.647059


In [17]:
# join two dataframes
df_field = df_1year.join(df_3year)
df_field.head()

,count_am_2018,count_md_2018,count_pm_2018,count_all_2018,count_am_1618,count_md_1618,count_pm_1618,count_all_1618
loc,,,,,,,,
1,1271.0,2933.0,4483.0,3220.6,1370.6,2814.833333,4512.666667,2989.294118
2,1749.0,1880.0,4935.5,3076.0,2029.6,2265.666667,5158.166667,3217.117647
3,2209.0,6639.0,9282.5,6810.4,2502.2,7508.333333,10915.833333,7238.588235
4,1648.0,1508.0,2787.0,2047.6,2061.8,1483.500000,2974.666667,2179.882353
5,1016.0,2513.0,3771.0,2716.8,1034.4,2504.000000,3588.833333,2454.647059


# Construct variables from CNN dataset
Each row in the cnn dataset generated from the previous notebook has the score of the detection of a person in a image. Each location has 18 images; each image has multiple (0 to many) rows for identified person in the image. The score's range is 0.2 to 1.0 since our cut-off probability was 0.2.

The next step is assign one number to each location so that we can compare the number estimated from CNN to the number from field count.

We may think of several different approaches to find one number representing pedestrian volume estimated from GSV images. 

1) count the number of detections not less than a certain threshold, say, 0.8.
  - if a location has total 3 detections above 0.8, then the location's pedestrian volume is 3.
  - pro. simple, intuitive.
  - con. we need to decide the threshold; information from less then the threshold goes away.
  
2) add up all scores from detections 
  - if a location has total 5 detections and there scores are 0.9, 0.85, 0.85, 0.5, 0.2, then the location's pedestrian volume is 3.3.
  - pro. we use more information.
  - con. don't know how much the information from low score is useful.
  
3) compromisation. add up all scores not less than a certain threshold
  - if a location has total 5 detections and there scores are 0.9, 0.85, 0.85, 0.5, 0.2 and the threshold is 0.5, then the location's pedestrian volume is 3.1.
    pro. we use more information with quality.
  - con. we need to decide the threshold.

We are going to construct 3 variables representing pedestrian volume of a location measured by faster RCNN on GSV images. 

  - 1) cnn_count: number of detections of which score is no less than 0.8
  - 2) cnn_sum_02: total sum of detection scores over 0.2 
  - 3) cnn_sum_05: total sum of detection scores over 0.5


In [19]:
df_cnn_long = pd.read_csv("../data/dataframe/nyc-cnn-exp.csv")\
    .rename(columns={'lid':'loc'})
# remove loc 102
df_cnn_long = df_cnn_long.drop(df_cnn_long[df_cnn_long['loc']==102].index)
df_cnn_long.head()

,index,image,score,loc
0,0,image_23_80.jpg,0.999439,23
1,0,image_23_80.jpg,0.999275,23
2,0,image_23_80.jpg,0.996823,23
3,0,image_23_80.jpg,0.991345,23
4,0,image_23_80.jpg,0.988284,23


In [20]:
df_cnn = pd.DataFrame({
    'cnn_count':df_cnn_long[df_cnn_long['score'] >= 0.8].groupby('loc').size(),
    'cnn_sum_02': df_cnn_long[df_cnn_long['score'] >= 0.2].groupby('loc').sum()['score'],
    'cnn_sum_05': df_cnn_long[df_cnn_long['score'] >= 0.5].groupby('loc').sum()['score'],
})
df_cnn = df_cnn.fillna(0)  # fill 0 where loc without count
df_cnn.head()

,cnn_count,cnn_sum_02,cnn_sum_05
loc,,,
1,43.0,58.994938,51.748607
2,5.0,11.921850,8.642439
3,97.0,166.952693,134.381194
4,8.0,17.379551,12.843091
5,39.0,67.678092,52.184549


In [21]:
len(df_cnn)

113

## Join Field count and CNN estimation

In [22]:
df = df_field.join(df_cnn)
print(len(df))
df.head()

113


,count_am_2018,count_md_2018,count_pm_2018,count_all_2018,count_am_1618,count_md_1618,count_pm_1618,count_all_1618,cnn_count,cnn_sum_02,cnn_sum_05
loc,,,,,,,,,,,
1,1271.0,2933.0,4483.0,3220.6,1370.6,2814.833333,4512.666667,2989.294118,43.0,58.994938,51.748607
2,1749.0,1880.0,4935.5,3076.0,2029.6,2265.666667,5158.166667,3217.117647,5.0,11.921850,8.642439
3,2209.0,6639.0,9282.5,6810.4,2502.2,7508.333333,10915.833333,7238.588235,97.0,166.952693,134.381194
4,1648.0,1508.0,2787.0,2047.6,2061.8,1483.500000,2974.666667,2179.882353,8.0,17.379551,12.843091
5,1016.0,2513.0,3771.0,2716.8,1034.4,2504.000000,3588.833333,2454.647059,39.0,67.678092,52.184549


In [23]:
df.to_csv("../data/dataframe/df_count.csv")